# Model Recapitation and Mutation

As introduced in the [previous tutorial](https://elissasoroj.github.io/shadie/notebooks/4-plantmodels/), `shadie` utilizes [recapitation](https://tskit.dev/pyslim/docs/latest/tutorial.html) to increase the efficiency of complex simulations that use alternation-of-generations models. Without it, these models are not as feasible to run. 

This means that, by deafult, `shadie` will run only the selected part of a simulation with SLiM when you use the function `model.run()`, and then the resulting `model.trees` file will need to be *recapitated* and *mutated* before the simulation is truly complete.

The use of alternation-of-generations (altgen) models can complicate the recapitation process because time in SLiM and time in [msprime](https://tskit.dev/msprime/docs/stable/intro.html), already confusing at the best of times, is even more mismatched by the altgen model architecture. 

In [3]:
import shadie
import pyslim
print("pyslim", pyslim.__version__)

pyslim 1.0.4


In [4]:
#set up and run a toy model
#make the chromosome
chrom = shadie.chromosome.explicit({
    (0, 200_000): shadie.NONCDS,
    (200_001, 400_000): shadie.EXON,
    (400_001, 600_000): shadie.INTRON,
    (600_001, 800_000): shadie.EXON,
    (800_001, 1_000_000): shadie.NONCDS,
})
chrom.draw();

#write the model
with shadie.Model() as recap_ex:
    recap_ex.initialize(
        chromosome=chrom, 
        sim_time=100,
        mutation_rate=1e-8, 
        recomb_rate=1e-9,
        file_out="recap_ex.trees",
    )
    recap_ex.reproduction.pteridophyte_homosporous(
        spo_pop_size=500, 
        gam_pop_size=1000,
    )

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="700.0px" height="150.0px" viewBox="0 0 700.0 150.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t602ad95394b6484f9d0407925c22e36d"> name: noncds
interval: (0, 200000)
ElementType: g3
is coding: False name: exon
interval: (200001, 400000)
ElementType: g1
is coding: True name: intron
interval: (400001, 600000)
ElementType: g2
is coding: True name: exon
interval: (600001, 800000)
ElementType: g1
is coding: True name: noncds
interval: (800001, 1000000)
ElementType: g3
is coding: False 0 200000 400000 600000 800000 1000000

In [7]:
f = open("script_ex.txt", "w")
f.write(recap_ex.script)
f.close()

In [3]:
recap_ex.run()

There are multiple functions in the `postsim` module to account for the different kinds of simulations you might run. Most users will likely utilize the `OneSim()` module which, as its name suggests, helps you process a single `shadie` simulation. We will focus on this function in this tutorial.

However, if you ran a specialized simulation you may need one of the other functions. A short description of the available functions and links to other tutorials:
- `.postsim.OneSim()`: process and analyze a "normal" `shadie` simulation. You did *not* use the `skip_neutral_mutations` parameter, or set the parameter to `skip_neutral_mutations=True`. You did not include a burn-in period in your simulation. 
- `.postsim.TwoSims()`: this is a specific function that helps you merge two separate simulations into one ancestral population at the beginning of the simulation then recapitate and mutate it. Conditions should be the same as above. This is most often used to simulate a population split, allowing you to perform certain analyses, such as the MK test. See the [demography tutorial]() for more detail and the [Quick Demo]() for another example of this.
- `.postsim.PureSlim()`: process and analyze an altered `shadie` simulation in which you simulated neutral mutations **and** simulated a burn-in (or loaded in a burn-in before continuing the simulation with `shadie` - an example of that can be found in [--]()). Usually, this means you utilized the parameter setting `skip_neutral_mutations=False`. In this case, you are loading the simulation purely to analyze the results. `shadie` will take care of the time unit mismatch that occurs due to the use of altgen simulations.
- `.postsim.PureSlim_TwoPops()`: a combination of `PureSlim()` and `TwoSims()`, this handles the case where multiple simulations were started from the same burn-in file and need to be merged so that they can be analyzed together. 

The postsim functions only need two parameters: the .trees file and the chromosome object. The chromosome is not necessary for running analyses, it is only used for plotting functions. 

In [5]:
#load the simulation into the post-sim module
post = shadie.postsim.OneSim("recap_ex.trees", chrom)

🌿 INFO | one_sim.py | shadie assumes sims were run without a burn-in and without neutral mutations and will recapitate (add ancestry simulation) and mutate (add neutral mutations) any loaded sims by default.
Current settings -
Recapitate: True
Add neutral mutations: True

🌿 INFO | one_sim.py | Simulation currently has 7 existing mutations of type(s) {2, 3}.
🌿 INFO | one_sim.py | Mutating using mutation rate: 1e-08. Keeping 53 existing mutations of type(s) {0, 2, 3}.


In [19]:
post.tree_sequence.population(2)

Population(id=2, metadata={'description': 'ancestral population simulated by msprime', 'name': 'ancestral'})

As you can see from the logger output, `shadie` will automatically recapitate and mutate the simulation and will report how many mutations it added. Mutations of type `0` are automatically defined as neutral by shadie, so you should always see no mutations of this type in the original simulation, and then the additional mutation type = 0 added after mutation. 

If you do not input any parameters, as in the example above, the function will attempt to recapitate and mutate the simulation using information from the metadata that `shadie` writes to the .trees file. If you did not use `shadie` to generate your model, or you altered it yourself in the SLiMgui, you may find that some of this metadata is missing (or worse, incorrect!). 

If metadata is missing you are able to enter it manually: 

In [3]:
#load the simulation into the post-sim module
post_manual = shadie.postsim.OneSim(
    trees_file="recap_ex.trees",
    chromosome=chrom,
    generations=500, #this is full lifecycles
    gens_per_lifecycle=2, #this indicates whether sim is altgen (=2) or not (=1)
    ancestral_Ne=500, #usually the same as the diploid population size
    mutation_rate=1e-8, 
    recomb_rate=1e-9,
)

🌿 INFO | one_sim.py | shadie assumes sims were run without a burn-in and without neutral mutations and will recapitate (add ancestry simulation) and mutate (add neutral mutations) any loaded sims by default.
Current settings -
Recapitate: True
Add neutral mutations: True

🌿 INFO | one_sim.py | Simulation currently has 7 existing mutations of type(s) {2, 3}.
🌿 INFO | one_sim.py | Mutating using mutation rate: 1e-08. Keeping 59 existing mutations of type(s) {0, 2, 3}.


As you can see, the results of the recapitation and mutation are very similar. `shadie` uses a random seed for the coalescent ancestry simulation and the mutation function, so every time it is run you will see slightly different results. If you want control over the seed you will need to run recapitate() and mutate() manually, then you can use the parameter `random_seed`. 

You can also control the behavior of the OneSim() function, for example you may not want it to recapitate (perform a coalescent ancestry simulation until a single ancestor is reached) or you may not want it to mutate (overlay neutral mutations that were not part of the original simulation).

In [5]:
#load the simulation into the post-sim module
post_norecap = shadie.postsim.OneSim("recap_ex.trees",
                             chrom, 
                             recapitate=False, #turn off recapitation
                             add_neutral_mutations=False) #turn off neutral mutations

🌿 INFO | one_sim.py | shadie assumes sims were run without a burn-in and without neutral mutations and will recapitate (add ancestry simulation) and mutate (add neutral mutations) any loaded sims by default.
Current settings -
Recapitate: False
Add neutral mutations: False



**Note**: If you set both `recapitate=False` and `add_neutral_mutations=False`, you are essentially creating the `PureSlim()` function. `PureSlim()` is a utility function, but even if you are running full SLiM simulations you can still keep using the `OneSim()` function to analyze those results if you are already comfortable with it. 

You can use the function `draw_tree_sequence()` to plot the treesequence if desired. If you do so, you will see that the original .trees file (loaded and not recapitated or mutated as `post_norecap`) only has the non-neutral mutations, while the recapitated and mutated object (loaded as `post`) has many additional neutral mutations. 

In [6]:
post_norecap.draw_tree_sequence(sample=25, seed=333);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="750.0px" height="425.0px" viewBox="0 0 750.0 425.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t7b5297d2a9124b24b44aa8e99814fb1f"> name: noncds
interval: (0, 200000)
ElementType: g3
is coding: False name: exon
interval: (200001, 400000)
ElementType: g1
is coding: True name: intron
interval: (400001, 600000)
ElementType: g2
is coding: True name: exon
interval: (600001, 800000)
ElementType: g1
is coding: True name: noncds
interval: (800001, 1000000)
ElementType: g3
is coding: False 0 200000 400000 600000 800000 1000000 idx: 0
interval: (0 - 985667)
tmrca: 201.0 idx: 1
interval: (985667 - 1000001)
tmrca: 201.0 p0-2 p0-7 p0-16 p0-18 p0-20 p0-17 p0-8 p0-19 p0-5 p0-11 p0-14 p0-12 p0-24 p0-22 p0-0 p0-23 p0-10 p0-13 p0-9 p0-15 p0-21 p0-1 p0-6 p0-3 p0-4 id: 0
pos: 0
time: 115
mtype: 3 p0-2 p0-7 p0-16 p0-18 p0-20 p0-17 p0-8 p0-19 p0-5 p0-11 p0-14 p0-12 p0-24 p0-22 p0-0 p0-23 p0-10 p0-9 p0-15 p0-21 p0-1 p0-6 p0-3 p0-4 p0-13 0 50 100 150 200 250

In [7]:
post.draw_tree_sequence(sample=25, seed=333);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="750.0px" height="425.0px" viewBox="0 0 750.0 425.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t6be7bacb91854bf88ec1809e2996d117"> name: noncds
interval: (0, 200000)
ElementType: g3
is coding: False name: exon
interval: (200001, 400000)
ElementType: g1
is coding: True name: intron
interval: (400001, 600000)
ElementType: g2
is coding: True name: exon
interval: (600001, 800000)
ElementType: g1
is coding: True name: noncds
interval: (800001, 1000000)
ElementType: g3
is coding: False 0 200000 400000 600000 800000 1000000 idx: 0
interval: (0 - 985667)
tmrca: 306.44 idx: 1
interval: (985667 - 1000001)
tmrca: 306.44 p0-2 p0-7 p0-16 p0-18 p0-20 p0-17 p0-8 p0-19 p0-5 p0-11 p0-14 p0-12 p0-24 p0-22 p0-0 p0-23 p0-10 p0-13 p0-9 p0-15 p0-21 p0-1 p0-6 p0-3 p0-4 id: 0
pos: 0
time: 51
mtype: 0 id: 1
pos: 1
time: 53
mtype: 0 id: 2
pos: 2
time: 14
mtype: 0 id: 3
pos: 3
time: 91
mtype: 0 id: 4
pos: 4
time: 42
mtype: 0 id: 5
pos: 5
time: 115
mtype: 3 p0-2 p0-7 p0-16 p0-18 p0-20 p0-17 p0-8 p0-19 p0-5 p0-11 p0-14 p0-12 p0-24 p0-22 p0-0 p0-23 p0-10 p0-9 p0-15 p0-21 p0-1 p0-6 p0-3 p0-4 p0-13 0 100 200 300

Sometimes a non-recapitated simulation cannot be plotted because there is no root - this indicates that there is more than one ancestor at the beginning of the simulation. If you instead plot the recapitated and mutated object (loaded as `post`) you will see a tree with a single root because the ancestry simulation has been performed. 

In [8]:
# the more samples you include, the more likely it is that multiple roots will exist in the tree
post_norecap.draw_tree_sequence(sample=125, seed=333);

⚠️ toytree | toytree_sequence:_get_toytree | tree has multiple (3) roots.


KeyError: None

See the next tutorial for more functionality of the `postsim` module for analysis